# Building Fast Queries

This project shows about the time assessing comparison between normal search and using better search and the reference dataset is laptop prices on Kaggle  as this [link](https://www.kaggle.com/ionaskel/laptop-prices)

First, import dataset and store header and the other data in separated variable

In [1]:
import csv
with open('laptops.csv') as file:
    file = list(csv.reader(file))
    header = file[0]
    rows = file[1:]
print(header)
for row in rows[:5]:
    print(row)

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']
['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', '1339']
['7287764', 'Apple', 'Macbook Air', 'Ultrabook', '13.3', '1440x900', 'Intel Core i5 1.8GHz', '8GB', '128GB Flash Storage', 'Intel HD Graphics 6000', 'macOS', '1.34kg', '898']
['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', '575']
['9722156', 'Apple', 'MacBook Pro', 'Ultrabook', '15.4', 'IPS Panel Retina Display 2880x1800', 'Intel Core i7 2.7GHz', '16GB', '512GB SSD', 'AMD Radeon Pro 455', 'macOS', '1.83kg', '2537']
['8550527', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 3.1GHz', '8GB', '256GB SSD',

## Inventory Class

</br>This class contains 6 functions as detail below:
1. **Initiary**
<p> This function works in intitial state. When the class is called, it will read file and separate header and data </p>
- **header (list)** : store header of dataset
- **rows (list)** : store data without header 
<p>and then, it will create helpful storage for the function support including </p>
- **id_to_row (dictionary)** : store id and price (used in 'get_laptop_from_id_fast' function)
- **prices (list)** : store only price (used in 'check_promotion_dollars_fast' function)
- **rows_by_price (list)** : sort the 'rows' list by price
2. **Get laptop id** 
<p>This function will search data from id in 'rows (list)' </p><br>
3. **Get laptop id faster**
<p>This functions works as same as section 2 but search in 'id_to_row (dictionary)' instead. </p><br>
4. **Check promotion from dollars**
<p>This function will search 1-2 laptops which suit with dollars promotion in 'rows (list)'. </p><br>
5. **Check promotion from dollars faster**
<p>This functions works as same as section 4 but search in 'price (list)' instead. </p><br>
6. **Find first laptop more expensive**
<p>This function will search laptop which suit with target price by using binary search </p><br>

In [2]:
class Inventory:
    
    #Initiary Function
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            file = list(csv.reader(file))
            
        #Store Header
        self.header = file[0]
        #Store Data
        self.rows = file[1:]
        #Change type of price (in the last column of each row) to Integer
        for row in self.rows:
            price = int(row[-1])
            row[-1] = price
        
        #Store id and price in dictionary
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row[1:]
            
        #Store only price in list    
        self.prices = []
        for row in self.rows:
            self.prices.append(row[-1])
            
        #Sort data by price
        def row_price(row):
            return row[-1]
        self.rows_by_price = sorted(self.rows, key=row_price)
    
    #Get laptop from id function
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows :
            if row[0] == laptop_id:
                return row
        return None
    
    #Get laptop from id faster function
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        else:
            return None
        
    #Check promotion dollars function
    def check_promotion_dollars(self, dollars):
        for row in self.rows :
            if row[-1] == dollars:
                return True
        for row1 in self.rows :
            for row2 in self.rows :
                if row1+row2 == dollars:
                    return True
        return False
    
    #Check promotion dollars fast function
    def check_promotion_dollars_fast(self, dollars):
        for price in self.prices:
            if price == dollars:
                return True
        for price1 in self.prices:
            for price2 in self.prices:
                if price1+price2 == dollars:
                    return True
        return False
    
    #Find first laptop more expensive function
    def find_first_laptop_more_expensive(self, target_price):
        range_start = 0
        range_end = len(self.rows_by_price)-1
        
        while range_start<range_end:
            range_middle = (range_end+range_start)//2
            price = self.rows_by_price[range_middle][-1]
            if price == target_price:
                return range_middle
            elif price < target_price:
                range_start = range_middle + 1
            else:
                range_end = range_middle - 1
        price = self.rows_by_price[range_start][-1]
        if price != target_price:
            return -1
        return range_start

### Initiary function

call Inventory class and check Intiary function work

In [3]:
data = Inventory('laptops.csv')

In [4]:
print(data.header)
print('The number of laptop: ',len(data.rows))

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']
The number of laptop:  1303


### Data get laptop function
test function by input id '3362737' and '3362736'

In [5]:
print(data.get_laptop_from_id('3362737'))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]


In [6]:
print(data.get_laptop_from_id('3362736'))

None


### Data get laptop fast function
test function by input id '3362737' and '3362736'

In [7]:
print(data.get_laptop_from_id_fast('3362737'))

['HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]


In [8]:
print(data.get_laptop_from_id_fast('3362736'))

None


### Compare how fast between two function

In this section, we will use random to create list which contains different 10000 ids. It could help to clearly see time difference between two function.

In [9]:
import time, random
ids = [str(random.randint(1000000, 9999999)) for _ in range(10000)]


total_time_no_dict = 0
for identifier in ids:
    start = time.time() #set begin time
    data.get_laptop_from_id(identifier) #first function
    end = time.time() #set end time
    total_time_no_dict += end-start #add how much time this function spends of each price
    
total_time_dict = 0
for identifier in ids:
    start = time.time() #set begin time
    data.get_laptop_from_id_fast(identifier) #second function
    end = time.time() #set end time
    total_time_dict += end-start #add how much time this function spends of each price
    

print('Time of first function: ',total_time_no_dict)
print('Time of second function: ',total_time_dict)

Time of first function:  0.9968557357788086
Time of second function:  0.004706144332885742


According to the result, we can conclude that creating new dictionary could be searched faster than using list which contains whole data.

### Data check promotion dollars function

test function by input price 1000 dollars and 442 dollars.

In [10]:
print(data.check_promotion_dollars(1000))

True


In [11]:
print(data.check_promotion_dollars(442))

False


### Data check promotion dollars fast function
test function by input price 1000 dollars and 442 dollars.

In [12]:
print(data.check_promotion_dollars_fast(1000))

True


In [13]:
print(data.check_promotion_dollars_fast(442))

False


### Compare how fast between two function
In this section, we will use random to create list which contains different 100 prices. It could help to clearly see time difference between two function.

In [14]:
prices = [random.randint(100,5000) for _ in range(100)]

total_time=0
for price in prices:
    start = time.time() #set begin time
    data.check_promotion_dollars(price) #first function
    end = time.time() #set end time
    total_time += end-start #add how much time this function spends of each price
    
total_time_list=0
for price in prices:
    start = time.time() #set begin time
    data.check_promotion_dollars_fast(price) #second function
    end = time.time() #set end time
    total_time_list += end-start #add how much time this function spends of each price

print('Time of first function: ',total_time)
print('Time of second function: ',total_time_list)

Time of first function:  22.48433756828308
Time of second function:  1.0614807605743408


According to the result, we can conclude that creating new list could be searched faster than using list which contains whole data.

### Data find fist laptop more expensive function

test function by input price 1000 dollars and 1000 dollars. It will return the number of row if it found but it not, it will return -1.

In [15]:
print(data.find_first_laptop_more_expensive(1000))

682


In [16]:
print(data.find_first_laptop_more_expensive(10000))

-1
